In [ ]:
"""
TODO:
1. add previous n_bookings as a feature
2. add number rooms booked previously as average
"""

In [1]:
import pandas as pd
import lightgbm
import category_encoders as ce
from sklearn.preprocessing import CategoricalEncoder,OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import datetime

In [2]:
train = pd.read_csv("train_5CLrC8b/train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [3]:
concated = pd.concat([train,test],ignore_index=True)

/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [4]:
concated['booking_date'] = pd.to_datetime(concated['booking_date'],dayfirst=True)
concated['checkin_date'] = pd.to_datetime(concated['checkin_date'],dayfirst=True)
concated['checkout_date'] = pd.to_datetime(concated['checkout_date'],dayfirst=True)

In [5]:
concated['booking_month'] = concated['booking_date'].apply(lambda x:x.month)
concated['booking_year'] = concated['booking_date'].apply(lambda x:x.year)
concated['booking_day'] = concated['booking_date'].apply(lambda x:x.day)

concated['checkin_month'] = concated['checkin_date'].apply(lambda x:x.month)
concated['checkin_year'] = concated['checkin_date'].apply(lambda x:x.year)
concated['checkin_day'] = concated['checkin_date'].apply(lambda x:x.day)

concated['checkout_month'] = concated['checkout_date'].apply(lambda x:x.month)
concated['checkout_year'] = concated['checkout_date'].apply(lambda x:x.year)
concated['checkout_day'] = concated['checkout_date'].apply(lambda x:x.day)

In [6]:
concated['booking_dayofweek'] = concated['booking_date'].apply(lambda x:x.dayofweek)

concated['checkin_dayofweek'] = concated['checkin_date'].apply(lambda x:x.dayofweek)

concated['checkout_dayofweek'] = concated['checkout_date'].apply(lambda x:x.dayofweek)

In [7]:
concated['season_holidayed_code_is_na'] = concated.season_holidayed_code.apply(
    lambda x: 1 if pd.isna(x) else 0)
concated['state_code_residence_is_na'] = concated.state_code_residence.apply(
    lambda x: 1 if pd.isna(x) else 0)
concated['season_holidayed_code'] = concated.season_holidayed_code.apply(
    lambda x: 0 if pd.isna(x) else x)
concated['state_code_residence'] = concated.state_code_residence.apply(
    lambda x: 0 if pd.isna(x) else x)

In [8]:
concated['actual_days_booked'] = concated.apply(
    lambda x: (x['checkout_date'] - x['checkin_date']).days, axis=1)

In [9]:
concated['n_rooms_booked'] = concated.apply(
    lambda x: x['roomnights'] / x['actual_days_booked'], axis=1)

In [10]:
concated['person_per_room'] = concated.apply(
    lambda x: (x['numberofadults'] + x['numberofchildren']) / x['n_rooms_booked'] if x['n_rooms_booked'] else 0,
    axis=1)

In [11]:
def count_weekend(row):
    checkin_date = row['checkin_date']
    actual_days_booked = row['actual_days_booked']
    stayed_in_weekend = 0
    for i in range(actual_days_booked):
        new_day = (checkin_date + datetime.timedelta(days=i)).day_name()
        if new_day == "Saturday" or new_day == "Friday" or new_day == "Sunday":
            stayed_in_weekend += 1
    return stayed_in_weekend

In [12]:
concated['n_weekend_days'] = concated.apply(count_weekend,axis=1)

In [13]:
concated['checkin_month']
seasons = []
for i in list(concated['checkin_month']):
    if i >=1 and i<4:
        seasons.append('w')
    if i>=4 and i<8:
        seasons.append('q')
    if i>=8 and i<11:
        seasons.append('e')
    if i>=11 and i<=12:
        seasons.append('t')
concated['ses']= seasons

In [14]:
concated.drop(['reservation_id','booking_date','checkin_date','checkout_date','memberid'],axis=1,inplace=True)

In [16]:

l=['member_age_buckets','cluster_code','reservationstatusid_code','resort_id','ses']

encoder = ce.OrdinalEncoder()
concated[l] = encoder.fit_transform(concated[l])

#encoder = OrdinalEncoder(dtype=np.int32,)
#concated[['memberh_age_buckets','cluster_code','reservationstatusid_code','resort_id']] = encoder.fit_transform(
#    concated[['member_age_buckets','cluster_code','reservationstatusid_code','resort_id']]
#)

In [ ]:
# # concated.to_csv("./concat_train_test_feat_engg.csv",index=False)
# from sklearn.decomposition import PCA
# x_ = concated
# x_pca_data = x_.drop(['amount_spent_per_room_night_scaled'],axis=1)
# pca = PCA(n_components=20)
# pca_ = pca.fit_transform(x_pca_data)


In [ ]:
# new_df = pd.DataFrame(pca_)
# new_df.head()

In [ ]:
#concated.drop(['intercept'],axis=1,inplace=True)
#concated.head()

In [17]:
concated

,amount_spent_per_room_night_scaled,booking_type_code,channel_code,cluster_code,main_product_code,member_age_buckets,numberofadults,numberofchildren,persontravellingid,reservationstatusid_code,...,booking_dayofweek,checkin_dayofweek,checkout_dayofweek,season_holidayed_code_is_na,state_code_residence_is_na,actual_days_booked,n_rooms_booked,person_per_room,n_weekend_days,ses
0,7.706428,1,3,1,1,1,2,0,46,1,...,3,3,4,0,0,1,1.000000,2.000000,0,1
1,6.662563,1,1,1,1,1,2,0,46,2,...,4,5,3,0,0,5,1.000000,2.000000,2,1
2,7.871602,1,1,2,1,1,2,0,47,2,...,2,6,3,0,0,4,1.000000,2.000000,1,2
3,5.344943,1,1,3,1,1,2,2,46,2,...,5,3,1,0,0,5,1.000000,4.000000,3,1
4,7.059346,1,1,3,1,1,2,0,46,2,...,2,0,5,0,0,5,1.000000,2.000000,1,3
5,8.167707,1,1,1,1,1,2,0,47,2,...,1,1,2,0,0,1,2.000000,1.000000,0,2
6,6.449838,1,3,4,1,1,2,0,46,2,...,2,2,3,0,0,1,1.000000,2.000000,0,2
7,7.046172,1,1,1,1,1,2,2,46,2,...,0,3,1,0,0,5,1.000000,4.000000,3,4
8,5.394203,1,1,4,1,1,2,0,45,2,...,5,6,5,0,0,6,1.000000,2.000000,2,4
9,6.561412,1,1,2,1,1,2,0,45,2,...,2,0,3,0,0,3,1.333333,1.500000,0,3


In [19]:
train = concated[:len(train)]
test = concated[-len(test):]

In [22]:
train.to_csv("pre_pro_train.csv",index=False)
test.to_csv("pre_pro_test.csv",index=False)

In [ ]:
all_predictions = []
all_mse = []
X = train#.drop(['amount_spent_per_room_night_scaled'], axis=1)
Y = y['amount_spent_per_room_night_scaled']
for i in range(10):
    print(i)
    trainX, testX, trainY, testY = train_test_split(X, Y,test_size=0.15)
    lgb_model = lightgbm.LGBMRegressor(n_estimators=1000)
    lgb_model.fit(
        trainX,
        trainY,
        eval_set=(testX, testY),
        eval_metric='rmse',
        early_stopping_rounds=100,verbose = 0 
    )
    predictions = lgb_model.predict(test, axis=1)
    test_pred = lgb_model.predict(testX)
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    all_mse.append(sqrt(mean_squared_error(testY, test_pred)))
    all_predictions.append(predictions)

In [ ]:
internal_pred = np.stack(internal_test_pred,axis=1).mean(axis=1)
final_predictions = np.stack(all_predictions,axis=1).mean(axis=1)

In [ ]:
sample_submission['amount_spent_per_room_night_scaled'] = final_predictions
sample_submission.to_csv(
    "poly_enc_1__pca_seson_included.csv",
    index=False)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(y_test_internal, pred_orig_test))


In [ ]:
np.array(all_mse).mean(axis=0)

In [ ]:
all_mse